### 03. 깊은 CNN으로 MNIST 분류하기

#### 1. 모델 이해하기
- 앞에서 공부한 1번 레이어와 2번 레이어는 동일하되, 새로운 합성곱층과 전결합층을 추가

1. 1번 레이어 : 합성곱층(Convolutional layer)
- 합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU + 맥스풀링(kernel_size=2, stride=2))

2. 2번 레이어 : 합성곱층(Convolutional layer)
- 합성곱(in_channel=32, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU + 맥스풀링(kernel_size=2, stride=2))

3. 3번 레이어(new) : 합성곱층(Convolutional layer)
- 합성곱(in_channel=64, out_channe=128, kernel_size=3, stride=1, padding=1) + 활성화함수 ReLU + 맥스풀링(kernel_size=2, stride=2, padding=1))

4. 4번 레이어(new) : 전결합층(Fully-Connected layer)
- 특성맵을 펼친다 : batch_size x 4 x 4 x 128 -> batch_size x 2048
- 전결합층(뉴런 625개) + 활성화함수 ReLU

5. 5번 레이어 : 전결합층(Fully-Connected layer)
- 전결합층(뉴런 10개) + 활성화함수 Softmax

-------------------------------------------------------------------------------------------------------------

#### 2. 깊은 CNN으로 MNIST 분류하기

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 된다. 

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda' : 
    torch.manual_seed_all(777)

학습에 사용될 파라미터 설정

In [27]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해 데이터셋을 정의

In [28]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

데이터 로더를 사용하여 배치 크기 지정

In [29]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

클래스로 모델 설계

In [43]:
class CNN(torch.nn.Module) : 
    def __init__(self) : 
        super(CNN, self).__init__()
        # 드롭아웃을 지원하기 위해 x 이외에 keep_prob 변수를 인수로 받아야 한다. 
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x) : 
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC(배치 사이즈를 고려 해야 하므로 맨 앞은 그래도 냅두고(배치자리), 나머지 곱)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

모델 정의

In [44]:
model = CNN().to(device)

비용함수와 옵티마이저 정의

In [45]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치의 수 출력

In [46]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


총 배치의 수는 600인데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개

In [ ]:
for epoch in range(training_epochs) : 
    avg_cost = 0
    
    # 미니 배치 단위로 꺼내온다. (X는 미니배치, Y는 레이블)
    for X, Y in data_loader : 
        # 이미지는 28x28 사이즈, no reshape
        # 레이블은 원-핫 인코딩됨
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.198674247
[Epoch:    2] cost = 0.0524641909
[Epoch:    3] cost = 0.0381815024
[Epoch:    4] cost = 0.0296787079
[Epoch:    5] cost = 0.0239549074
[Epoch:    6] cost = 0.0213318095
[Epoch:    7] cost = 0.0174029246
[Epoch:    8] cost = 0.0154676111
[Epoch:    9] cost = 0.0128467595
[Epoch:   10] cost = 0.0132595785
[Epoch:   11] cost = 0.010218841
[Epoch:   12] cost = 0.0100592244


테스트

In [ ]:
# 학습을 진행하지 않을 것이므로 
with torch.no_grad() : 
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = modle(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())